# Milestone 1

## Import

In [1]:
import json
import os
import requests
import pandas as pd
import zipfile

## Download the Data

### Download from API

In [2]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
output_dir = "../data/"

In [3]:
resp = requests.get(url)

file_list = json.loads(resp.content)['files']

Note: The cell below takes around 1.5 minutes to run. 

In [4]:
%%time
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
for file in file_list:
    if file['name'] != "data.zip":
        continue
    output_name = os.path.join(output_dir, file['name'])
    if os.path.exists(output_name):
        print(f"{file['name']} already exists!")
    else:
        print(f"Downloading {file['name']} to {output_dir}...")
        file_resp = requests.get(file['download_url'])
        with open(output_name, "wb") as f:
            for chunk in file_resp.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
        print("Download finished")

data.zip already exists!
CPU times: user 615 µs, sys: 849 µs, total: 1.46 ms
Wall time: 1.71 ms


### Unzip data

In [5]:
raw_csv_dir = os.path.join(output_dir, "raw")

In [6]:
%%time
with zipfile.ZipFile(output_name, "r") as zf:
    zf.extractall(raw_csv_dir)

CPU times: user 21.5 s, sys: 6.79 s, total: 28.3 s
Wall time: 39.8 s


## Combine CSV Files

In [7]:
dir_files = os.listdir(raw_csv_dir)
output_df = pd.DataFrame()

In [ ]:
%%time
# Exclude MACOSX_ & observed_daily_rainfall_SYD.csv
for fname in dir_files:
    if fname[-8:] != "_NSW.csv":
        continue

    model_name = fname.split('_')[0]

    df = pd.read_csv(os.path.join(raw_csv_dir, fname), index_col=0)
    df['model'] = model_name
    output_df = pd.concat([output_df, df])
    del df

output_df.to_csv(os.path.join(output_dir, "combined.csv"))

In [ ]:
%%sh
du -sh ../data/combined.csv

In [ ]:
output_df.shape

In [ ]:
output_df.head()

In [ ]:
del output_df

### Comparison

| Team Member| Operating System | RAM        | Processor                        | Is SSD | CPU Time   | Wall Time |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    | 7min 54s   | 8min 10s  |
| Kyle       |Windows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |      | 8min 15s  |
| LG         | Big Sur 11.5.2  | 8 GB | 2.9 GHz Dual-Core Intel Core i5 |  Yes      |  9min 58s    | 10min 45s  | 
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    | 6min 33s   | 6min 34s  |

**Discussion:**
> The running time among the team members are roughly the same. However, we could not see a clear relationship between the RAM/CPU with the running time. In reality, there could be many external conditions affecting the running time such as number of process running concurrently. The memory management algorithm among various operating systems might also be accountable. Note that `%%time` could not report the CPU time on Windows.

## Load the Combined CSV and simple EDA

In [13]:
%reset -f

In [14]:
import pandas as pd

### Approach 1: Load Only Columns We want

Let's try `value_counts()` on `model` column. Let's compare how long it takes when we only load in `model` column vs all columns.

In [15]:
%%time
use_cols = ["model"]
df = pd.read_csv("../data/combined.csv", usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 36.1 s, sys: 4.35 s, total: 40.5 s
Wall time: 44.4 s


In [16]:
%%time
df = pd.read_csv("../data/combined.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 1min 9s, sys: 24.6 s, total: 1min 33s
Wall time: 1min 40s


#### Comparison - Load Only Columns We Want

| Team Member| Operating System | RAM        | Processor        | Is SSD | CPU Time (model)   | Wall Time (model) |  CPU Time (all) | Wall Time (all) |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|:---------------:|:---------------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    |    32.7 s  |   33.1 s  |   1min 31s      | 1min 48s        |
| Kyle       | Windows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |            | 46.5s|     |1min 23s
| LG         | Big Sur 11.5.2 | 8 GB | 2.9 GHz Dual-Core Intel Core i5 | Yes | 40.5 s | 44.4 s     | 1min 33s     | 1min 40s
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    |  29.2s     | 29.7s| 1min 03s      | 1min 03s

**Discussion:**
> We can observe that when only `model` column is loaded, there is a significant reduction in Wall Time.

### Approach 2

In [17]:
%reset -f
import pandas as pd

In [18]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("../data/combined.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 1min 10s, sys: 10.8 s, total: 1min 21s
Wall time: 1min 25s


#### Comparison - Load in Chunk

| Team Member| Operating System | RAM        | Processor        | Is SSD | CPU Time (chunk)   | Wall Time (chunk) |  CPU Time (all) | Wall Time (all) |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|:---------------:|:---------------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    |    1min 8s |  1min 10s |   1min 31s      | 1min 48s        |
| Kyle       | Windows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |            | 1min 14s|     |1min 23s
| LG         | Big Sur 11.5.2 | 8 GB  | 2.9 GHz Dual-Core Intel Core i5 | Yes | 1min 21s           | 1min 25s    | 1min 33s | 1min 40s
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    |  1min          | 1min 01s    |  1min 03s    | 1min 03s

**Discussion:**
> We can observe that there is a reasonable reduction in Wall Time when we run `value_counts()` by loading in smaller chunks, 10 million rows each in our case. It seems that there is not a significant reduction in time because RAM size is large enough to load in all the data in all.

## Simple EDA in R

In [21]:
%reset -f
%load_ext rpy2.ipython

In [37]:
%%R
library(dplyr)
library(arrow)

R[write to console]: Error in library(dplyr) : there is no package called ‘dplyr’




Error in library(dplyr) : there is no package called ‘dplyr’


RInterpreterError: Failed to parse and evaluate line 'library(dplyr)\nlibrary(arrow)\n'.
R error message: 'Error in library(dplyr) : there is no package called ‘dplyr’'

### 6.1 Using arrow table and pyra 

In [33]:
pip install rpy2_arrow

  Using cached rpy2-arrow-0.0.5.tar.gz (5.9 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2-arrow: filename=rpy2_arrow-0.0.5-py3-none-any.whl size=6616 sha256=b142246306cc0599484020c7a0061f73a67d6e247a5ac94bb0cf7a9d6afd4538
  Stored in directory: /Users/giangnguyen/Library/Caches/pip/wheels/c2/11/c9/eaf77c0f9ab594dae4f51e3a293bd45a5300d54fcc885cedc3
Successfully built rpy2-arrow
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

PackageNotInstalledError: The R package "arrow" is not installed.

In [35]:
%%time
# loading csv file to pyarrow dataset and then converting to pyarrow table
dataset = ds.dataset("../data/combined.csv", format="csv")
table = dataset.to_table()

CPU times: user 21.8 s, sys: 5.69 s, total: 27.5 s
Wall time: 24.3 s


In [36]:
%%time
# converting pyarrow table to rarrow table
r_table = pyra.converter.py2rpy(table)

NameError: name 'pyra' is not defined

In [11]:
%%time
%%R -i r_table
# referenced lecture 2 note
# performing simple EDA to count the number of each model
start_time <- Sys.time()
library(dplyr)
counts <- r_table %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 5.146541 secs
CPU times: user 4.75 s, sys: 1 s, total: 5.75 s
Wall time: 5.18 s


### 6.2 Using feather

In [12]:
%%time
# creating feather file from above table
import pyarrow.feather as feather
feather.write_feather(table, '../data/combined.feather')

CPU times: user 5 s, sys: 1.34 s, total: 6.34 s
Wall time: 3.25 s


In [13]:
%%time
%%R
# transfering the feather file to R
r_table_feather <- arrow::read_feather("../data/combined.feather")

CPU times: user 4.43 s, sys: 4.2 s, total: 8.63 s
Wall time: 2.62 s


In [14]:
%%time
%%R
# performing the same EDA as above
start_time <- Sys.time()
library(dplyr)
counts <- r_table_feather %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 4.336046 secs
CPU times: user 3.85 s, sys: 526 ms, total: 4.38 s
Wall time: 4.36 s


### Discussion
We chose to experiment with Feather with the following reasons:  
- According to various sources, Feather is faster than Parquet to read into Arrow Table.
- Compared to vanila serialization/deserialization process as performed above, Feather indeed performed much faster.
- Using Feather file also saved the storage, as the file generated is only 1.1 GB.
- One reservation is that Feather is not recommended for long-term data storage.